# Cvičení 4a

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat LDLT rozkladem.

## LDLT rozklad

LDLT rozklad je vhodný pro symetrické matice. Lze jej popsat následujícím pseudokódem:

```
n = size(A)
L = I, D = 0

v(1) = A(1, 1)
D(1, 1) = v(1)
L(2:n, 1) = A(2:n, 1)/v(1)

for j = 2 to n do
    for i = 1 to j - 1 do
        v(i) = L(j, i)D(i, i)
    end

    v(j) = A(j, j) - L(j, 1:j-1)v(1:j-1)
    D(j, j) = v(j)
    L(j+1:n, j) = (A(j+1:n, j) - L(j+1:n, 1:j-1)v(1:j-1)) / v(j)
end
```

V následující části implementujete LDLT rozklad a využijete ho k řešení soustavy se symetrickou maticí.

In [47]:
# ÚKOL: Doplňte následující kód

import numpy as np

def my_ldlt(A):
    """
    Vypočítá LDLT rozklad symetrické matice.
    Vstup: A - A symetrická matice
    Výstup: L, D - Matice dekompozice takové, že platí A = L*D*L.T
    """
    m, n = A.shape

    # Zkontrolujme, že je matice čtvercová
    if m != n:
        raise ValueError('Matice musí být čtvercová!')

    L = np.eye(m)
    D = np.zeros((m, m))
    v = np.zeros(m)

    # doplňte
    v[0] = A[0, 0]
    D[0, 0] = v[0]
    L[1:n, 0] = A[1:n, 0] / v[0]

    for j in range(1, n):
        for i in range(j-1):
            v[i] = L[j, i] * D[i, i]
        
        v[j] = A[j, j] - L[j, 0:j-1] @ v[0:j-1]
        D[j, j] = v[j]
        L[j+1:n, j] = (A[j+1:n, j] - L[j+1:n, 0:j-1] @ v[0:j-1]) / v[j]

    return L, D

A = np.array([[1, 2, 3],
              [2, 4, 5],
              [3, 5, 6]])
L, D = my_ldlt(A)
print(L)
print(D)
print(L @ D @ L.T)

[[1.   0.   0.  ]
 [2.   1.   0.  ]
 [3.   1.25 1.  ]]
[[ 1.  0.  0.]
 [ 0.  4.  0.]
 [ 0.  0. -3.]]
[[ 1.    2.    3.  ]
 [ 2.    8.   11.  ]
 [ 3.   11.   12.25]]


In [86]:
# Vygenerujeme náhodnou symetrickou matici
A = np.random.rand(5, 5)

A = A + A.transpose()
print(A)

[[0.35750669 1.9073024  0.81948986 1.28212895 1.09431212]
 [1.9073024  0.62475576 0.41908817 1.18695134 1.3749263 ]
 [0.81948986 0.41908817 0.83265372 0.44719005 1.09689383]
 [1.28212895 1.18695134 0.44719005 1.64093348 1.17120989]
 [1.09431212 1.3749263  1.09689383 1.17120989 1.54411037]]


In [51]:
# Otestujeme, že váš kód vrací správný výsledek
L, D = my_ldlt(A)
AA = L@D@L.transpose()
print(AA-A)

[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.23468205  0.32283565  0.30388092  0.09822814]
 [ 0.          0.32283565  0.7139469   0.2682272   0.48864003]
 [ 0.          0.30388092  0.2682272   0.07759442 -0.46456855]
 [ 0.          0.09822814  0.48864003 -0.46456855  0.00819028]]


Známe-li rozklad symetrické matice $\mathsf{A} = \mathsf{L}\mathsf{D}\mathsf{L}^T$, můžeme jej využít k řešení soustavy $\mathsf{A}\mathbf{x} = \mathbf{b}$, kterou převedeme na 
$$\mathsf{L}\mathsf{D}\mathsf{L}^T\mathbf{x} = \mathbf{b}$$
a vyřešíme vhodnými substitucemi s využitím dopředné a zpětné substituce.

Rozmyslete si, jak takové substituce budou vypadat a v následující části vyřešte náhodnou soustavu.

In [55]:
# ÚKOL: Zkopírujte z minulého cvičení

def fsubst(L, b):
    m, n = L.shape
    x = np.zeros(n)

    x[0] = b[0] / L[0, 0]

    for i in range(1, m):
        suma = 0
        for j in range(i):
            suma += L[i, j] * x[j]

        x[i] = (b[i] - suma)/L[i, i]

    return x

In [56]:
# ÚKOL: Zkopírujte z minulého cvičení

def bsubst(U, b):
    m, n = U.shape
    x = np.zeros(n)

    x[-1] = b[-1] / U[-1, -1]

    for i in range(m-1, -1, -1):
        suma = 0
        for j in range(m-1, i, -1):
            suma += U[i, j] * x[j]
        
        x[i] = (b[i] - suma)/U[i, i]
    
    return x

In [87]:
import scipy
import scipy.linalg

# ÚKOL: Doplňte následující buňku a vyřešte soustavu rovnic Ax = b se symetrickou náhodnou maticí A a
# náhodným vektorem pravé strany b. Využijte vaše metody fsubst a bsubst
b = np.random.rand(5)

# doplňte řešení soustavy Ax = b pomoci LDLT rozkladu
L, D, P = scipy.linalg.ldl(A)
AA = L @ D @ L.T
print(L)
print(AA - A)

y = fsubst(L, b)
y_s_vlnkou = np.zeros(b.size)
for i in range(b.size):
    y_s_vlnkou[i] = y[i] / D[i, i]
x = bsubst(L.T, y_s_vlnkou)

print(x)
print(A @ x)
print(b)

[[ 1.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.        ]
 [ 0.08415617  0.4138848   1.          0.          0.        ]
 [ 0.42843167  0.59191545 -0.25747419  1.          0.        ]
 [ 0.56779965  0.46731968  0.73824847  0.0020676   1.        ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.22044605e-16
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.55111512e-17  0.00000000e+00
  -2.22044605e-16]
 [ 0.00000000e+00 -5.55111512e-17  0.00000000e+00 -5.55111512e-17
  -2.22044605e-16]
 [-2.22044605e-16  0.00000000e+00 -5.55111512e-17 -2.22044605e-16
  -2.22044605e-16]
 [ 0.00000000e+00 -2.22044605e-16  0.00000000e+00 -2.22044605e-16
  -2.22044605e-16]]
[-2.61126275 -0.26027715 -3.63567634  0.84933321  5.52216311]
[2.72255366 1.93393818 1.16079813 2.57856114 2.31822744]
[0.20811758 0.82362841 0.4896525  0.8440876  0.37166191]


In [88]:
# Porovnáme vaše řešení a řešení pomocí numpy
print(x)
print(np.linalg.solve(A, b))

[-2.61126275 -0.26027715 -3.63567634  0.84933321  5.52216311]
[-2.77140227 -1.54858106 -3.63567634  0.84933321  5.52216311]
